# Utils 

In [13]:
from transformers import RobertaTokenizer, RobertaConfig, RobertaModel
import torch 
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

import torch.nn as nn

## Parameters

In [14]:
def count_parameter():
    model_name_or_path="microsoft/unixcoder-base"
    unixcoder_model = RobertaModel.from_pretrained(model_name_or_path, output_attentions=True,output_hidden_states=True) 
    result["base"] = {}
    result["base"]["param"] = round((count_parameters(unixcoder_model)+classifier_number)/ 1000000, 2)
    for name, param in unixcoder_model.named_parameters():
        if name.startswith("pooler.dense."): # choose whatever you like here
            param.requires_grad = False

    for name, param in unixcoder_model.named_parameters():
        if name.startswith("embeddings"): # choose whatever you like here
            param.requires_grad = False
    result[0]= {}
    result[0]["param"] = round((count_parameters(unixcoder_model)+classifier_number)/ 1000000, 2)

    for i in range(11):
        for name, param in unixcoder_model.named_parameters():
            if name.startswith("encoder.layer.%d."%i): # choose whatever you like here
                param.requires_grad = False
        result[i+1]= {}
        result[i+1]["param"] = round((count_parameters(unixcoder_model)+classifier_number)/ 1000000, 2)

In [15]:
def count_parameter_change_ratio():
    for key in result:
        param_decrease_ratio = 1- result[key]["param"]/result["base"]["param"]
        result[key]["param_decrease_ratio"] = round(param_decrease_ratio*100,2)

## Time Cost

In [16]:
import json
import os
import numpy as np
def read_json_file(filename):
    with open(filename, 'r') as fp:
        data = fp.readlines()
    if len(data) == 1:
        data = json.loads(data[0])
    else:
        data = [json.loads(line) for line in data]
    return data
def obtain_second(given_str):
#     given_str='7m10s'
    h,m,s= 0,0,0
    if "h" in given_str:
        h_split = given_str.split("h")
        h= int(h_split[0])
        given_str = h_split[-1]
    if "m" in given_str:
        m_split = given_str.split("m")
        m= int(m_split[0])
        given_str = m_split[-1]
    if "s" in given_str:
        s_split = given_str.split("s")
        s= int(s_split[0])
    return 3600*h+60*m+s
def time_format(seconds):
    m, s = divmod(seconds, 60)
    h, m = divmod(m, 60)
    return "%dh%02dm%02ds" % (h, m, s)

In [17]:
def process_data(data_dir):
#     data_dir="../DownstreamTasks/clone-detection/times"
    filename="each_epoch.jsonl"
    times= read_json_file( os.path.join(data_dir,filename))
    filename="coverage_epoch.jsonl"
    coverage_epoch =read_json_file( os.path.join(data_dir,filename))
    for key in result:
        result[key]["each_epoch"] = times[str(key)]
        given_str = times[str(key)]
        time_decrease_ratio = 1- obtain_second(result[key]["each_epoch"])/obtain_second(result["base"]["each_epoch"])
        result[key]["each_epoch_decrease_ratio"] = round( time_decrease_ratio*100,2)
        result[key]["coverage_epoch"]= time_format( obtain_second(given_str)*coverage_epoch[str(key)] ) 
        time_decrease_ratio = 1- obtain_second(result[key]["coverage_epoch"])/obtain_second(result["base"]["coverage_epoch"])
        result[key]["coverage_epoch_decrease_ratio"] = round( time_decrease_ratio*100,2)

## Performance

In [18]:
def get_result(root_path, lang, filename):
    if lang == None:
        filename_lang_model_name = os.path.join(root_path, filename)
    else:
        filename_lang_model_name = os.path.join(root_path,lang, filename)
#     print(filename_lang_model_name)
    time_stamp = os.listdir(filename_lang_model_name)[0]
    result_filename = os.path.join(filename_lang_model_name, time_stamp, "result.jsonl" )
    data = read_json_file(result_filename)
    return data

def process_peformance(root_path,lang):
#     lang="python"
#     root_path = "../DownstreamTasks/clone-detection/saved_models/partial_freezing"
    diff_paramter_results = {}
    diff_values = [-1,0,1,2,3,4,5,6,7,8,9,10,11]
    if task == "code-summarization": 
        diff_values = [-1,0,1,2,3,4,5,6,7,8,9,10,11]
        diff_paramter_result=read_json_file(os.path.join(root_path,lang,"results.jsonl"))
        for value in diff_values  :
            if value == -1:
                value = "base"
            diff_paramter_results [value] =  diff_paramter_result[str(value)]
        print(diff_paramter_results)
    else:
        for value in diff_values  :
            filename =  "freeze_bottom_%d_layers"%int(value)
            diff_paramter_results [value] = get_result(root_path, lang, filename)
        diff_paramter_results ["base"] = diff_paramter_results [-1]
    for key in result:
        result[key]["performance"] = diff_paramter_results[key]
        result[key]["performance_decrease_ratio"] ={}
        for metric in diff_paramter_results[key]:
            performance_decrease_ratio = 1- result[key]["performance"][metric]/ result["base"]["performance"][metric]
            result[key]["performance_decrease_ratio"][metric] = round( performance_decrease_ratio*100,2)

In [19]:
import pandas as pd
#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', None)
#设置value的显示长度为100，默认为50
pd.set_option('max_colwidth',100)

# Downstream Tasks

## Code Search

In [20]:

classifier_number = 0
task = "code-search"
result ={}
count_parameter()
count_parameter_change_ratio()
data_dir="../DownstreamTasks/%s/times"%task
process_data(data_dir)
lang="python"
root_path = "../DownstreamTasks/%s/saved_models/partial_freezing"%task
process_peformance(root_path,lang)
result_df = pd.DataFrame(result).T
result_df.columns = ['Param', 'Ratio', 'Each Epoch', 'Ratio', 'Coverage','Ratio', 'Performance', 'Ratio']
model_variants =["Telly-%s"%item for item in result.keys()]
model_variants[0] = "Base"
result_df.index= model_variants
print(result_df)

           Param  Ratio Each Epoch  Ratio  Coverage  Ratio  \
Base      125.93    0.0     17m14s    0.0  2h35m06s    0.0   
Telly-0    85.05  32.46     15m49s   8.22  2h06m32s  18.42   
Telly-1    77.97  38.08     14m53s  13.64  1h59m04s  23.23   
Telly-2    70.88  43.71     14m06s  18.18  1h52m48s  27.27   
Telly-3    63.79  49.34     13m12s   23.4  0h39m36s  74.47   
Telly-4     56.7  54.97     12m12s  29.21  0h48m48s  68.54   
Telly-5    49.62   60.6     11m42s  32.11  0h35m06s  77.37   
Telly-6    42.53  66.23     10m50s  37.14  0h32m30s  79.05   
Telly-7    35.44  71.86      9m57s  42.26  0h29m51s  80.75   
Telly-8    28.35  77.49      9m04s  47.39  0h36m16s  76.62   
Telly-9    21.26  83.12      8m14s  52.22  0h24m42s  84.07   
Telly-10   14.18  88.74      7m10s  58.41  0h21m30s  86.14   
Telly-11    7.09  94.37      6m21s  63.15  0h19m03s  87.72   

                                                             Performance  \
Base       {'R@1': 0.612, 'R@5': 0.838, 'R@10': 0.889, 

## Code Clone

In [21]:
class RobertaClassificationHead(nn.Module):
    """Head for sentence-level classification tasks."""

    def __init__(self):
        super().__init__()
        hidden_size= 768
        self.dense = nn.Linear(hidden_size*2, hidden_size)
        self.dropout = nn.Dropout(0.1)
        self.out_proj = nn.Linear(hidden_size, 2)

    def forward(self, x):
        x = x.reshape(-1,x.size(-1)*2)
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x
classifier = RobertaClassificationHead()
classifier_number = count_parameters(classifier )
result ={}
count_parameter()
count_parameter_change_ratio()
data_dir="../DownstreamTasks/clone-detection/times"
process_data(data_dir)
lang=None
root_path = "../DownstreamTasks/clone-detection/saved_models/partial_freezing"
process_peformance(root_path,lang)
result_df = pd.DataFrame(result).T
result_df.columns = ['Param', 'Ratio', 'Each Epoch', 'Ratio', 'Coverage','Ratio', 'Performance', 'Ratio']
model_variants =["Telly-%s"%item for item in result.keys()]
model_variants[0] = "Base"
result_df.index= model_variants
print(result_df)

           Param  Ratio Each Epoch  Ratio  Coverage  Ratio  \
Base      127.11    0.0     20m21s    0.0  1h41m45s    0.0   
Telly-0    86.24  32.15     19m29s   4.26  1h17m56s  23.41   
Telly-1    79.15  37.73     18m36s    8.6  1h14m24s  26.88   
Telly-2    72.06  43.31     17m34s  13.68  1h10m16s  30.94   
Telly-3    64.97  48.89     15m20s  24.65  1h01m20s  39.72   
Telly-4    57.88  54.46     14m28s  28.91  0h57m52s  43.13   
Telly-5     50.8  60.03     13m25s  34.07  1h07m05s  34.07   
Telly-6    43.71  65.61     12m35s  38.17  0h50m20s  50.53   
Telly-7    36.62  71.19     11m44s  42.34  0h58m40s  42.34   
Telly-8    29.53  76.77     10m41s   47.5  0h53m25s   47.5   
Telly-9    22.45  82.34      9m55s  51.27  0h29m45s  70.76   
Telly-10   15.36  87.92      8m51s  56.51  0h35m24s  65.21   
Telly-11    8.27  93.49      8m00s  60.69  0h32m00s  68.55   

                                                             Performance  \
Base      {'eval_recall': 0.95, 'eval_precision': 0.95,

## Code Summarization

In [22]:

classifier_number = 0
task = "code-summarization"
result ={}
count_parameter()
count_parameter_change_ratio()
data_dir="../DownstreamTasks/%s/times"%task
process_data(data_dir)
lang="python"
root_path = "../DownstreamTasks/%s/saved_models/partial_freezing"%task
process_peformance(root_path,lang)
result_df = pd.DataFrame(result).T
result_df.columns = ['Param', 'Ratio', 'Each Epoch', 'Ratio', 'Coverage','Ratio', 'Performance', 'Ratio']
model_variants =["Telly-%s"%item for item in result.keys()]
model_variants[0] = "Base"
result_df.index= model_variants
print(result_df)

{'base': {'bleu': 19.15, 'meteor': 17.26, 'rouge': 41.51, 'cider': 1.54}, 0: {'meteor': 17.32, 'rouge': 41.5, 'cider': 1.53, 'bleu': 19.19}, 1: {'meteor': 17.33, 'rouge': 41.57, 'cider': 1.54, 'bleu': 19.21}, 2: {'meteor': 17.3, 'rouge': 41.51, 'cider': 1.54, 'bleu': 19.17}, 3: {'meteor': 17.26, 'rouge': 41.5, 'cider': 1.54, 'bleu': 19.16}, 4: {'meteor': 17.26, 'rouge': 41.53, 'cider': 1.54, 'bleu': 19.13}, 5: {'meteor': 17.26, 'rouge': 41.55, 'cider': 1.54, 'bleu': 19.18}, 6: {'meteor': 17.35, 'rouge': 41.78, 'cider': 1.56, 'bleu': 19.36}, 7: {'meteor': 17.28, 'rouge': 41.74, 'cider': 1.56, 'bleu': 19.37}, 8: {'meteor': 17.26, 'rouge': 41.69, 'cider': 1.55, 'bleu': 19.34}, 9: {'meteor': 17.22, 'rouge': 41.33, 'cider': 1.53, 'bleu': 19.18}, 10: {'meteor': 17.18, 'rouge': 41.23, 'cider': 1.53, 'bleu': 19.11}, 11: {'meteor': 17.2, 'rouge': 41.27, 'cider': 1.53, 'bleu': 19.1}}
           Param  Ratio Each Epoch  Ratio  Coverage  Ratio  \
Base      125.93    0.0     22m44s    0.0  1h53m40s

## Code Generation

In [23]:

classifier_number = 0
task = "code-generation"
result ={}
count_parameter()
count_parameter_change_ratio()
data_dir="../DownstreamTasks/%s/times"%task
process_data(data_dir)
lang=None
root_path = "../DownstreamTasks/%s/saved_models/partial_freezing"%task
process_peformance(root_path,lang)
result_df = pd.DataFrame(result).T
result_df.columns = ['Param', 'Ratio', 'Each Epoch', 'Ratio', 'Coverage','Ratio', 'Performance', 'Ratio']
model_variants =["Telly-%s"%item for item in result.keys()]
model_variants[0] = "Base"
result_df.index= model_variants
print(result_df)

           Param  Ratio Each Epoch  Ratio  Coverage  Ratio  \
Base      125.93    0.0     12m25s    0.0  4h08m20s    0.0   
Telly-0    85.05  32.46     11m43s   5.64  3h42m37s  10.36   
Telly-1    77.97  38.08     11m07s  10.47  3h20m06s  19.42   
Telly-2    70.88  43.71     10m32s  15.17  2h06m24s   49.1   
Telly-3    63.79  49.34     09m52s  20.54  2h08m16s  48.35   
Telly-4     56.7  54.97     09m15s   25.5  1h51m00s   55.3   
Telly-5    49.62   60.6     08m44s  29.66  1h53m32s  54.28   
Telly-6    42.53  66.23     08m13s  33.83  1h46m49s  56.99   
Telly-7    35.44  71.86     07m41s  38.12  1h39m53s  59.78   
Telly-8    28.35  77.49     07m08s  42.55  1h32m44s  62.66   
Telly-9    21.26  83.12     06m42s  46.04  1h27m06s  64.93   
Telly-10   14.18  88.74     06m03s  51.28  1h12m36s  70.77   
Telly-11    7.09  94.37     05m22s  56.78  1h09m46s  71.91   

                          Performance                         Ratio  
Base      {'BLEU': 33.82, 'EM': 17.4}      {'BLEU': 0.0, 'EM'

## Code Completion

In [24]:

classifier_number = 0
task = "code-completion"
result ={}
count_parameter()
count_parameter_change_ratio()
data_dir="../DownstreamTasks/%s/times"%task
process_data(data_dir)
lang=None
root_path = "../DownstreamTasks/%s/saved_models/partial_freezing"%task
process_peformance(root_path,lang)
result_df = pd.DataFrame(result).T
result_df.columns = ['Param', 'Ratio', 'Each Epoch', 'Ratio', 'Coverage','Ratio', 'Performance', 'Ratio']
model_variants =["Telly-%s"%item for item in result.keys()]
model_variants[0] = "Base"
result_df.index= model_variants
print(result_df)

           Param  Ratio Each Epoch  Ratio  Coverage  Ratio  \
Base      125.93    0.0     04m12s    0.0  0h37m48s    0.0   
Telly-0    85.05  32.46     03m59s   5.16  0h31m52s   15.7   
Telly-1    77.97  38.08     03m47s   9.92  0h18m55s  49.96   
Telly-2    70.88  43.71     03m38s  13.49  0h32m42s  13.49   
Telly-3    63.79  49.34     03m29s  17.06  0h20m54s  44.71   
Telly-4     56.7  54.97     03m18s  21.43  0h26m24s  30.16   
Telly-5    49.62   60.6     03m07s  25.79  0h09m21s  75.26   
Telly-6    42.53  66.23     02m57s  29.76  0h11m48s  68.78   
Telly-7    35.44  71.86     02m48s  33.33  0h14m00s  62.96   
Telly-8    28.35  77.49     02m39s   36.9  0h13m15s  64.95   
Telly-9    21.26  83.12     02m28s  41.27  0h12m20s  67.37   
Telly-10   14.18  88.74     02m17s  45.63  0h09m08s  75.84   
Telly-11    7.09  94.37     02m06s   50.0  0h06m18s  83.33   

                               Performance                             Ratio  
Base       {'EM': 20.4, 'Edit sim': 51.92}      {'EM